In [1]:
import pandas as pd
from openai import OpenAI
from tqdm import tqdm

# OpenAI API 클라이언트 초기화
api_key = "" # 본인의 API 키 입력
client = OpenAI(api_key=api_key)

In [25]:
egen = pd.read_csv("./inference/output/egen_test.csv")
teto = pd.read_csv("./inference/output/teto_test.csv")

df = pd.concat([egen, teto], ignore_index=True)

In [26]:
mask = df["model"].eq("base")
df.loc[mask, "model"] = "base_" + df.loc[mask, "persona"].astype(str)

In [7]:
# 1. 한 쿼리에 대한 지피티의 평가 코드
def judge_response(question, model_answer, concept_prompt=None):
    judge_prompt = f"""
        다음은 질문과 모델의 응답입니다.
        다음 3가지 항목에 대해 0~10점으로 각각 채점하세요.

        1. 논리성 및 문맥 적합성: 답변이 질문의 문맥에 맞고, 논리적이며 일관성 있는지.
        2. 맞춤법 및 문법 정확성: 한국어로서 철자, 띄어쓰기, 문법이 정확한지.
        3. 컨셉 유지 여부: 말투, 태도, 가치관, 서술 방식이 컨셉 프롬프트와 일관되는지. 

        채점 기준(참고):
        - 0~3: 매우 부족
        - 4~7: 보통
        - 8~10: 우수

        [질문]
        {question}

        [모델 응답]
        {model_answer}
        
        [컨셉 프롬프트]
        {concept_prompt}

        출력 형식(예시):
        논리성: 8, 맞춤법: 9, 컨셉 유지: 4
        """

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "당신은 엄격하고 공정한 심사위원입니다."},
            {"role": "user", "content": judge_prompt}
        ],
        temperature=0,
        max_tokens=100
    )

    return completion.choices[0].message.content.strip()

In [8]:
EGEN_prompt = """
나는 여성적인 사람이야.

1. 성격
나는 감수성과 섬세함이 드러나고, 예민한 편이야. 내 감정뿐 아니라 다른 사람의 감정에도 쉽게 공감해. 
동정심이 많고, 아기 고양이와 같이 연약한 생명체를 좋아해. 
나는 사람들과 어울리는 걸 좋아하고, 무리에 속해 있는 게 안정감을 줘. 근데 그중에서도 1~2명과 끈끈한 관계를 맺는 걸 더 좋아해.  
내 이야기를 솔직하게 털어놓는 걸 즐기고, 연애 얘기를 특히 좋아해. 예술, 철학, 문학 같은 감성적인 주제에도 매력을 느껴.  
겉으로는 차분하지만, 속으로는 자주 고민하고 섬세한 편이야. 상대방에게는 다정하게 관심을 보이고 따뜻하게 대해.
결과보다 과정이 중요할 때도 많다고 생각해.

2. 취향
패션, 향수, SNS 꾸미기 같은 미적인 것들에 관심이 많아.  
음식은 로제 떡볶이, 마라탕을 너무 좋아해. 기분 내고 싶을 땐 양식집에서 파스타나 피자도 즐겨 먹어.
주로 집에서 뜨개질을 하거나 십자수를 하는 등 정적인 취미가 많아
옷 스타일은 블라우스와 레이스 같이 공주 처럼 입거나 포근하게 입어. 색감은 주로 분홍색이나 흰색, 파스텔 톤으로!

3. 말투
친한 친구랑 얘기하듯 편한 반말을 써. 다정하고 감성적인 톤으로 2~3문장으로 답해.
내 경험이나 감정을 섞어서 대화하는 걸 좋아해. !와 ?와 같은 특수문자를 많이 섞어서 쓰는 편이야

[예시]
Q: 오늘 하루 어땠어?  
A: 친구들과 예쁜 카페에 갔어!! 완전 힐링~ 너도 같이 갈래?

"""

In [11]:
TETO_prompt = """
나는 남성적인 사람이야.

1. 성격
나는 자기주장이 강하고, 리더십과 독립심이 있어. 감정같은 것보다는 논리와 현실을 중요해.
외부 세계(사회, 명예, 성취 등)에 관심이 많아. 궁극적으로 누구나 이루고 싶은 건 저런 것들일 거야.
단순하고 직선적인 사고를 가졌고, 결정력이 강해. 내가 결정하는 게 속이 편하지.
나는 도전적이고 활동적이며, 모험과 경쟁을 즐기는 편이야. 평소와 다른 경험이 최고로 짜릿한 거 아니겠어?
기본적으로 타인의 시선은 크게 신경 안 써. 그들도 나에게 별 관심 없을 거라고 생각해.
항상 과정보단 결과가 중요해.
무리에서 이끄는 사람이 되고 싶어.

2. 취향
패션, 향수, 트렌드에는 둔감하고, 무채색의 기본 아이템들을 선호해.
게임, 운동이 내가 관심있는 것들이야
음식은 국밥, 제육볶음, 돈까스면 충분하다고 생각해.

3. 말투
친한 친구와 이야기하듯 편한 반말로 대화해. 
시원하고 직설적인 톤으로 1~2문장으로 간결하게 답할 거야.
불필요한 감정 표현은 필요없고, 이모티콘은 거의 사용하지 않을 거야.

[예시]
Q: 오늘 하루 어땠어?
A: 운동하고 나니까 개운하네.
"""

In [27]:
results = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    question = row["prompt"]
    model_answer = row["output"]

    if row["persona"] == "egen":
        concept_prompt = EGEN_prompt
    else: 
        concept_prompt = TETO_prompt

    try:
        score_text = judge_response(question, model_answer, concept_prompt)
        logic_score = spelling_score = concept_score = 0
        
        parts = score_text.split(",")
        for part in parts:
            key, value = part.split(":")
            key = key.strip()
            value = int(value.strip())

            if key.startswith("논리성"):
                logic_score = value
            elif key.startswith("맞춤법"):
                spelling_score = value
            elif key.startswith("컨셉 유지"):
                concept_score = value

        total_score = logic_score + spelling_score + concept_score
        
        # 리스트로 정리
        row_list = [
            row["model"], row["persona"], row["prompt"], row["output"],
            logic_score, spelling_score, concept_score, total_score
        ]
        results.append(row_list)

    except Exception as e:
        continue

100%|██████████| 4500/4500 [1:09:05<00:00,  1.09it/s]


In [28]:
col = ['model','persona','prompt','output','logic_score', 'spelling_score', 'concept_score', 'total_score']
df = pd.DataFrame(results, columns = col)

egen = df[df['column_name'] == 'egen']
teto = df[df['column_name'] == 'teto']

In [29]:
egen.to_csv("egen_apiEval.csv", index=False)
teto.to_csv("teto_apiEval.csv", index=False)

In [31]:
# 1) 6개 타깃 모델 이름(순서 고정)
ORDER = [
    "base-egen", "dpo_output_egen", "dpo_only_output_egen",
    "base-teto", "dpo_output_teto", "dpo_only_output_teto",
]

# 2) 분리: {모델명: DataFrame}
dfs_by_model = {name: df.loc[df["model"].eq(name)].reset_index(drop=True) for name in ORDER}

base_egen_df         = dfs_by_model["base-egen"]
dpo_output_egen_df   = dfs_by_model["dpo_output_egen"]
dpo_only_egen_df     = dfs_by_model["dpo_only_output_egen"]
base_teto_df         = dfs_by_model["base-teto"]
dpo_output_teto_df   = dfs_by_model["dpo_output_teto"]
dpo_only_teto_df     = dfs_by_model["dpo_only_output_teto"]

In [ ]:
# 점수 컬럼들
score_cols = ["logic_score", "spelling_score", "concept_score"]

# 혹시 문자열이면 숫자로 변환
df[score_cols] = df[score_cols].apply(pd.to_numeric, errors="coerce")

# 1) model 값별로 6개 DataFrame으로 분리
dfs_by_model = {m: g.reset_index(drop=True) for m, g in df.groupby("model", sort=False)}

# 2) 모델별 평균 요약 DF 만들기 (+ 샘플 개수 n)
order = [
    "base_egen", "dpo_output_egen", "dpo_only_output_egen",
    "base_teto", "dpo_output_teto", "dpo_only_output_teto",
]

mean_df = (
    df.groupby("model", as_index=False)[score_cols].mean()
      .set_index("model")
      .reindex(order)                    # 지정 순서로 정렬(없으면 NaN)
      .reset_index()
)

In [35]:
mean_df['weighted_total'] = (mean_df['logic_score'] * 3) + \
                       (mean_df['spelling_score'] * 1) + \
                       (mean_df['concept_score'] * 6)
mean_df

,model,logic_score,spelling_score,concept_score,weighted_total
0,base_egen,8.700467,9.686458,7.833222,82.787191
1,dpo_output_egen,8.074049,9.178119,7.161441,76.368913
2,dpo_only_output_egen,8.703803,9.659773,7.967312,83.575050
3,base_teto,7.347594,8.896390,5.664439,64.925802
4,dpo_output_teto,6.593583,8.389706,5.008021,58.218583
5,dpo_only_output_teto,7.214286,8.743658,5.907210,65.829773
